MODEL 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# Baca dataset dari file CSV
data = pd.read_excel('foodd.xlsx')  

# Mengisi nilai yang hilang dengan 0
data.fillna(0, inplace=True)

# Membaca atribut gizi sebagai fitur
features = data[['Kalori', 'Protein', 'Karbo', 'Lemak', 'Serat']].values

# Normalisasi fitur
features_norm = (features - features.mean(axis=0)) / features.std(axis=0)

# Membaca atribut rekomendasi sebagai target
target = data['Makanan'].astype('category').cat.codes

# Membagi data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(features_norm, target, test_size=0.2, random_state=42)

# Membangun model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(data['Makanan'].unique()), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Melatih model
model.fit(features_norm, target, epochs=10, batch_size=1)



Epoch 1/10
982/982 [==============================] - 3s 3ms/step - loss: 7.0631
Epoch 2/10
982/982 [==============================] - 3s 3ms/step - loss: 6.7462
Epoch 3/10
982/982 [==============================] - 3s 3ms/step - loss: 6.1425
Epoch 4/10
982/982 [==============================] - 2s 2ms/step - loss: 5.4833
Epoch 5/10
982/982 [==============================] - 2s 2ms/step - loss: 4.9355
Epoch 6/10
982/982 [==============================] - 3s 3ms/step - loss: 4.4326
Epoch 7/10
982/982 [==============================] - 3s 3ms/step - loss: 4.0159
Epoch 8/10
982/982 [==============================] - 2s 2ms/step - loss: 3.6495
Epoch 9/10
982/982 [==============================] - 2s 2ms/step - loss: 3.3413
Epoch 10/10
982/982 [==============================] - 3s 3ms/step - loss: 3.0883


In [ ]:
# Evaluasi model dengan data pengujian
loss = model.evaluate(X_test, y_test, verbose=0)

print("Loss: ", loss)


Loss:  2.640801191329956


In [ ]:
# Fungsi untuk merekomendasikan makanan berdasarkan data gizi pengguna
def recommend_food(nutrition_data, allergies=[]):
    # Normalisasi data gizi pengguna
    nutrition_data_norm = (nutrition_data - features.mean(axis=0)) / features.std(axis=0)

    # Memprediksi kelas makanan
    prediction = model.predict(np.array([nutrition_data_norm]))

    # Mendapatkan indeks kelas dengan probabilitas tertinggi
    predicted_class_index = np.argmax(prediction)

    # Mendapatkan nama makanan yang direkomendasikan
    recommended_food = data.loc[data['Makanan'].astype('category').cat.codes == predicted_class_index, 'Makanan'].values

    if len(recommended_food) > 0:
        recommended_food = recommended_food[0]
        # Pengecualian jika memiliki alergi terhadap makanan tertentu
        if recommended_food in allergies:
            return "Anda memiliki alergi terhadap makanan ini. Silakan coba makanan lain."
        else:
            return recommended_food
    else:
        return "Tidak ada makanan yang cocok dengan kebutuhan gizi Anda."

# Fungsi untuk menghitung BMR (Basal Metabolic Rate)
def calculate_bmr(weight, height, age, gender):
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    return bmr


In [ ]:
# Contoh penggunaan
weight = 57  # Ganti dengan berat badan pengguna dalam kilogram
height = 155  # Ganti dengan tinggi badan pengguna dalam sentimeter
age = 21  # Ganti dengan usia pengguna
gender = 'female'  # Ganti dengan jenis kelamin pengguna ('male' atau 'female')
allergies = ['Telur', 'Kacang']  # Ganti dengan daftar alergi pengguna, kosongkan jika tidak ada

bmr = calculate_bmr(weight, height, age, gender)
nutrition_data = [bmr, 100, 80, 120, 20]  # Gunakan BMR sebagai kalori makanan yang direkomendasikan

recommended_food = recommend_food(nutrition_data, allergies)
print("Makanan yang direkomendasikan: ", recommended_food)

1/1 [==============================] - 0s 38ms/step
Makanan yang direkomendasikan:   Soto Banjar  
